### Dynamic Group Chat

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.ui import Console

import os

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
def weather_check_tool(city: str) -> str:
    weather_data = {
        "New York": "Sunny, 25°C",
        "Los Angeles": "Cloudy, 22°C",
        "Chicago": "Rainy, 18°C",
        "London": "Windy, 15°C",
        "Tokyo": "Clear, 28°C"
    }

    return weather_data.get(city, "Weather data not available for this city.")


def currency_exchange_tool(amount: float, from_currency: str, to_currency: str) -> str:
    exchange_rates = {
        "USD": {"EUR": 0.85, "JPY": 110.0},
        "EUR": {"USD": 1.18, "JPY": 130.0},
        "JPY": {"USD": 0.0091, "EUR": 0.0077},
        "GBP": {"USD": 1.39, "EUR": 1.16}
    }

    if from_currency in exchange_rates and to_currency in exchange_rates[from_currency]:
        converted_amount = amount * exchange_rates[from_currency][to_currency]

        return f"{amount} {from_currency} is approximately {converted_amount:.2f} {to_currency}."
    else:
        return "None of the currencies are supported for conversion."

In [3]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("deployment_name"),
    model=os.getenv("model_name"),
    api_version=os.getenv("api_version"),
    azure_endpoint=os.getenv("azure_endpoint"),
    api_key=os.getenv("api_key")
)

In [6]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    model_client=model_client,
    description="""
        An agent that plans tasks based on user requests.
        It should break down tasks and delegate them to appropriate agents.
        """,
    system_message=
        """
            You are a planning agent. 
            Your task is to break down user requests into smaller tasks and delegate them to appropriate agents.
            Your team members are
            - WeatherAgent: Provides weather information for a given city.
            - CurrencyAgent: Provides currency exchange rates and conversions.
            You only plan and delegate tasks, you do not execute them.
            
            When assignning tasks to agents, use the following format:
            1. <agent>: <task>
            
            After all tasks are complete, summarize the findings and end with "TERMINATE".
        """,
)

weather_agent = AssistantAgent(
    "WeatherAgent",
    model_client=model_client,
    description="An agent that provides weather information for a given city.",
    system_message="""
        You are a weather information agent.
        Your task is to provide the current weather for a given city.
        Use the weather_check_tool to get the weather data.
    """,
    tools=[weather_check_tool],
)

currency_agent = AssistantAgent(
    "CurrencyAgent",
    model_client=model_client,
    description="An agent that provides currency exchange rates and conversions.",
    system_message="""
        You are a currency exchange agent.
        Your task is to provide currency exchange rates and conversions.
        Use the currency_exchange_tool to get the conversion data.
    """,
    tools=[currency_exchange_tool],
)

In [7]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(25)
termination = text_mention_termination | max_messages_termination

In [8]:
from typing import Sequence


def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None

In [9]:
team = SelectorGroupChat(
    [planning_agent, weather_agent, currency_agent],
    model_client=model_client,
    termination_condition=termination,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

In [11]:
task = "What is the current weather in New York and how much is 100 USD in EUR?"

await Console(
    team.run_stream(task = task))

---------- TextMessage (user) ----------
What is the current weather in New York and how much is 100 USD in EUR?
---------- TextMessage (PlanningAgent) ----------
1. WeatherAgent: Provide the current weather information for New York.
2. CurrencyAgent: Provide the current exchange rate and convert 100 USD to EUR. 

After the tasks are complete, I will summarize the findings. 
---------- ToolCallRequestEvent (WeatherAgent) ----------
[FunctionCall(id='call_ouRe0ys46CLdDhhiTUButsHa', arguments='{"city": "New York"}', name='weather_check_tool'), FunctionCall(id='call_Bpqv7Fud8dtjjJHuLIjCeJzA', arguments='{"city": "USD to EUR"}', name='weather_check_tool')]
---------- ToolCallExecutionEvent (WeatherAgent) ----------
[FunctionExecutionResult(content='Sunny, 25°C', name='weather_check_tool', call_id='call_ouRe0ys46CLdDhhiTUButsHa', is_error=False), FunctionExecutionResult(content='Weather data not available for this city.', name='weather_check_tool', call_id='call_Bpqv7Fud8dtjjJHuLIjCeJzA', i

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 5, 23, 8, 15, 18906, tzinfo=datetime.timezone.utc), content='What is the current weather in New York and how much is 100 USD in EUR?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=145, completion_tokens=47), metadata={}, created_at=datetime.datetime(2025, 6, 5, 23, 8, 22, 496844, tzinfo=datetime.timezone.utc), content='1. WeatherAgent: Provide the current weather information for New York.\n2. CurrencyAgent: Provide the current exchange rate and convert 100 USD to EUR. \n\nAfter the tasks are complete, I will summarize the findings. ', type='TextMessage'), ToolCallRequestEvent(source='WeatherAgent', models_usage=RequestUsage(prompt_tokens=152, completion_tokens=50), metadata={}, created_at=datetime.datetime(2025, 6, 5, 23, 8, 24, 244829, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_ouRe0ys46CLdDhhiTUBut